In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

from epw import epw
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import plotly.express as px
import random
import seaborn as sns
from energyPlus.weather.weather import *

In [ ]:
data, climate_zones = getWeatherData()

In [ ]:
fig, ax = plt.subplots()
for zone in climate_zones:
    zoneData = data[data["ClimateZone"]==zone]
    ax.plot(zoneData.index, zoneData["Dry Bulb Temperature"], label=zone)
ax.legend()

In [ ]:
chosenWeatherProperties = {}
chosenData = []
toPlot = True
N = 10000
n = 0
for n in range(N):
    weatherProperties, dataSampled = sampleVentWeather(data, climate_zones, runDays=7, dt=3600, plot=toPlot, coolingThreshold=24, coolingDegBase=21, keep="CDDs")
    for key, value in weatherProperties.items():
        if n == 0:
            chosenWeatherProperties[key] = [value]
        else:
            chosenWeatherProperties[key].append(value)
    chosenData.append(dataSampled)
    toPlot = False


In [ ]:
weather = pd.DataFrame(chosenWeatherProperties)
weather.columns = ["Climate Zone", "Month", "CDDs", "NCDDs", "Average Wind Speed [m/s]", "Days Checked"]
weather["Total Days"] = weather["Days Checked"].sum()
print(weather["Total Days"].values[0])

weights = None
plotWeatherHists(weather, weights, ["Month", "Average Wind Speed [m/s]"])
plotWeatherHists(weather, weights, ["CDDs", "NCDDs"])
plotWeatherHists(weather, weights, ["Climate Zone", "Days Checked"])

In [ ]:
plotWeatherHists(weather, 'CDDs', ["Month", "Average Wind Speed [m/s]"], yearScaling = 52.1429 / weather.shape[0])
plotWeatherHists(weather, 'NCDDs', ["Month", "Average Wind Speed [m/s]"], yearScaling = 52.1429 / weather.shape[0])

In [ ]:
plotCZMap(weather, weights="NCDDs", vmax=None, colormap = "PuBu")
plotCZMap(weather, weights="CDDs", vmax=1800, colormap = "PuBu")

In [ ]:
radiation_keys = ["Extraterrestrial Horizontal Radiation",
    "Extraterrestrial Direct Normal Radiation" ,
    "Horizontal Infrared Radiation Intensity" ,
    "Global Horizontal Radiation" ,
    "Direct Normal Radiation" ,
    "Diffuse Horizontal Radiation" ]
fig = px.line(data, x = data.index, y = radiation_keys)
fig.show()

In [ ]:
fig = px.line(data, x = data.index, y = [
    "Horizontal Infrared Radiation Intensity",
    "Global Horizontal Radiation",
    "Total Sky Radiation",
    ])
fig.show()

fig = px.line(data, x = data.index, y = [
    "Equivalent Sky Temperature",
    "Dry Bulb Temperature",
    ])
fig.show()